In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
del_df = pd.read_csv('../input/ipl-data-set/deliveries.csv')
match= pd.read_csv('../input/ipl-data-set/matches.csv')

In [ ]:
match.shape

In [ ]:
match.head()

In [ ]:
del_df.shape

In [ ]:
del_df.head(5)

# Check for null values

In [ ]:
match.isnull().sum()

In [ ]:
match.rename(columns={'Season':'season'},inplace=True)

# Check venues of null cities

In [ ]:
match[match['city'].isnull()==True]['venue']

All cities with null have venue as Dubai,so replace it with Dubai
Replace winner with null values wiht Draw
Maximum number of Umpire3 are null so drop that column
Replace player of the match null values with Unknown

In [ ]:
match.city.fillna('Dubai',inplace=True)
match.winner.fillna('Draw',inplace=True)
match.drop('umpire3',axis=1,inplace=True)
match.fillna('Unknown',inplace=True)

Check For Deliveries dataset

In [ ]:
del_df.isnull().sum()

It has only 3 columns

Replace all null values with 0


In [ ]:
del_df.fillna('0',inplace=True)

Basic Stats


In [ ]:
print('Total Seasons:',len(match.season.unique()))
print('Total Matches Played:',match.id.count())
print('Teams:',match.team1.unique())
print('Venues:',match.venue.unique())


As it can be seen, There are same teams and venues with different names(Dupliactes).

Replace them in both DataFrames.


In [ ]:
match.replace({'Rising Pune Supergiants':'Rising Pune Supergiant'},inplace=True)
match.replace({'Punjab Cricket Association IS Bindra Stadium, Mohali':'Punjab Cricket Association Stadium, Mohali'},inplace=True)
del_df.replace({'Rising Pune Supergiants':'Rising Pune Supergiant'},inplace=True)


In [ ]:
match.iloc[[match['win_by_runs'].idxmax()]][['season','team1','team2','win_by_runs','player_of_match','winner']]

Mumbai Indians has won the match with highest margin.

In [ ]:
match.iloc[[match['win_by_wickets'].idxmax()]][['season','team1','team2','win_by_wickets','player_of_match','winner']]

Kolkalta Knight Riders has wont the match with maximum wickets.

# Toss Decisions

In [ ]:
toss_decisions=match['toss_decision'].value_counts().reset_index()
plt.figure(figsize=(8,8))
sns.barplot(x='index',y='toss_decision',data=toss_decisions)
plt.xlabel('Toss-Decision',fontsize=15)
plt.ylabel('Count',fontsize=15)
plt.title('Toss-Decision Count',size=20)
plt.show()


In [ ]:
toss_wins_per_team=match['toss_winner'].value_counts().reset_index()
plt.figure(figsize=(8,8))
toss_plot=sns.barplot(x='index',y='toss_winner',data=toss_wins_per_team)
plt.xticks(rotation=90)
plt.xlabel('Teams',fontsize=15)
plt.ylabel('Toss Count',fontsize=15)

plt.title('Toss Wins Counts',size=20)
for p in toss_plot.patches:
    toss_plot.annotate(format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))


Mumbai Indians has won the toss maximum number of times.

# Matches Per Venue

In [ ]:
matches_per_venue=match['venue'].value_counts().reset_index()
plt.figure(figsize=(8,8))
venue_plot=sns.barplot(x='index',y='venue',data=matches_per_venue[:15])
plt.xticks(rotation=90)
plt.xlabel('Venue',fontsize=15)
plt.ylabel('Matches Count',fontsize=15)

plt.title('Matches Per Venue Counts',size=20)
for p in venue_plot.patches:
    venue_plot.annotate(format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))


Maximum number if matches are held at Eden Gardens.

# Bat first and Ball first wins comparison per venue

In [ ]:
batfirst_wins=match[match.win_by_runs>1].groupby('venue').apply(lambda x:x['win_by_runs'].count()).reset_index(name='BatFirst_Wins')
ballfirst_wins=match[match.win_by_wickets>1].groupby('venue').apply(lambda x:x['win_by_wickets'].count()).reset_index(name='BallFirst_Wins')
Innings_venue=batfirst_wins.merge(ballfirst_wins,on='venue').sort_values(['BatFirst_Wins','BallFirst_Wins'],ascending=False)
Innings_venue.plot.bar(x='venue',y=['BatFirst_Wins','BallFirst_Wins'],rot=90,figsize=(15,9),title='BatFirst/BallFirst Wins Per Venue')
plt.show()

# Toss And Match wins comparison per team

In [ ]:
toss_match_wins_per_team=match[match.toss_winner==match.winner].groupby('winner').apply(lambda x:x['winner'].count()).reset_index(name='Toss_Match_Wins')
toss_wins_per_team=match['toss_winner'].value_counts().reset_index(name='toss_wins')
toss_wins_per_team.rename(columns={'index':'winner'},inplace=True)
toss_match_win=toss_wins_per_team.merge(toss_match_wins_per_team,on='winner')
toss_match_win['Percetage']=(toss_match_win.Toss_Match_Wins/toss_match_win.toss_wins)*100
plt.figure(figsize=(10,7))
toss_mat_plot=sns.barplot(x='winner',y='Percetage',data=toss_match_win)
plt.xticks(rotation=90)
plt.xlabel('Teams',fontsize=15)
plt.ylabel('Match Wins',fontsize=15)

plt.title('Toss VS Match Wins',size=20)
for p in toss_mat_plot.patches:
    toss_mat_plot.annotate("%.2f" % p.get_height(), (p.get_x()*1.005, p.get_height()*1.005),  xytext=(0, 10), textcoords='offset points')


Delhi Capitals has high chances of winning the match when it wins the toss

**Matches Count Per Season**

In [ ]:
matches_per_season=match.groupby('season')['id'].count().reset_index()
plt.figure(figsize=(7,5))
season_plot=sns.barplot(x='season',y='id',data=matches_per_season,palette=sns.color_palette('winter'))
plt.xticks(rotation=90)
plt.xlabel('Season',fontsize=15)
plt.ylabel('Matches',fontsize=15)

plt.title('Matches Per Season',size=20)
for p in season_plot.patches:
    season_plot.annotate(format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))


**Let's try to find something on merging both datasets.**

In [ ]:
runs=match[['id','season']].merge(del_df,left_on='id',right_on='match_id',how='left').drop('id',axis=1)

**Runs Distribution Per Season**

In [ ]:
runs_per_season=runs.groupby('season')['total_runs'].sum().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(x='season',y='total_runs',data=runs_per_season,marker="o")
plt.xticks(rotation=90)
plt.xlabel('Season',fontsize=15)
plt.ylabel('Runs',fontsize=15)

plt.title('Runs Per Season',size=20)

plt.show()

Higest Runs Scored in IPL 2013


**Let's figure out boundaries per season**

In [ ]:
fours_season=runs.groupby('season')['batsman_runs'].agg(lambda x:(x==4).sum()).reset_index(name='Fours')
sixes_season=runs.groupby('season')['batsman_runs'].agg(lambda x:(x==6).sum()).reset_index(name='Sixes')
boundaries_per_season=fours_season.merge(sixes_season,on='season')
boundaries_per_season.set_index('season')[['Sixes','Fours']].plot(marker='o')
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel('Season',fontsize=15)
plt.ylabel('Count',fontsize=15)

plt.title("4's and 6's Per Season",size=20)
plt.show()

Higest number of sixes were hit in 2018 and fours in 2013


In [ ]:
fours=runs.groupby('batsman')['batsman_runs'].agg(lambda x:(x==4).sum()).reset_index(name='Fours')
sixes=runs.groupby('batsman')['batsman_runs'].agg(lambda x:(x==6).sum()).reset_index(name='Sixes')
Batsman=fours.merge(sixes,on='batsman')
six_hitter=Batsman.iloc[Batsman['Sixes'].idxmax()][['batsman']]
four_hitter=Batsman.iloc[Batsman['Fours'].idxmax()][['batsman']]
print('Maximum Six Hitter',six_hitter)
print('Maximum Four Hitter',four_hitter)
Batsman=Batsman.sort_values(by=['Sixes','Fours'],ascending=False)

Batsman[:20].plot.barh(x='batsman',y=['Sixes','Fours'],figsize=(7,7),title='Fours and Sixes Per Batsman')
plt.show()

In [ ]:
runs_per_season_per_team= runs.pivot_table(index=['season'],columns='batting_team',values='total_runs',aggfunc=sum)
runs_per_season_per_team.fillna(0,inplace=True) #replace nan values with 0
runs_per_season_per_team.plot(marker='o')
fig=plt.gcf()
fig.set_size_inches(15,6)
plt.legend(loc='upper right')
plt.xlabel('Season',fontsize=15)
plt.ylabel('Count',fontsize=15)

plt.title("Runs Per Season Per Team",size=20)
plt.show()

Now let's see the wins of teams per venue

In [ ]:
team_wins_per_venue=match.groupby(['winner','venue']).apply(lambda x:x['venue'].count()).reset_index(name='Win_Counts').sort_values(by=['winner','Win_Counts'],ascending=False)
    
def venue_wins_plot(team):
    team_per_venue=team_wins_per_venue[team_wins_per_venue['winner']==team][:15]
    plt.figure(figsize=(7,7))
    plt.pie(team_per_venue['Win_Counts'],labels=team_per_venue['venue'],autopct='%1.1f%%')
    title='Wins of '+team+' per Venue(top 10)'
    plt.title(title,size=20)
venue_wins_plot('Mumbai Indians')

Mumbai Indians Has won maximum matches at Wankhede Stadium and lowest at Sawai Mansingh Stadium



# Team Comparison

In [ ]:
def head_to_head_plot(team1,team2):
    mt1=match[((match['team1']==team1)|(match['team2']==team1))&((match['team1']==team2)|(match['team2']==team2))]
    sns.countplot(x='season', hue='winner',data=mt1,palette=sns.color_palette("pastel"))
    title=team1+' VS '+team2
    plt.xlabel('season',fontsize=15)
    plt.ylabel('Wins',fontsize=15)
    plt.title(title,size=20)
    plt.xticks(rotation='vertical')
    leg = plt.legend( loc = 'upper left')
    fig=plt.gcf()
    fig.set_size_inches(10,6)
    plt.show()

head_to_head_plot('Mumbai Indians','Chennai Super Kings')



# Score Distribution Per Innings

In [ ]:
score_dist=runs.groupby(['match_id','inning','batting_team'])['total_runs'].sum().reset_index()
Inning1_score=score_dist[score_dist['inning']==1]
Inning2_score=score_dist[score_dist['inning']==2]
plt.figure(figsize=(7,5))
sns.boxplot(x='batting_team',y='total_runs',data=Inning1_score)
plt.xlabel('Runs',fontsize=15)
plt.ylabel('Teams',fontsize=15)
plt.title('Inning 1 Scores',size=20)
plt.xticks(rotation=90)
plt.show()
plt.figure(figsize=(7,5))
sns.boxplot(x='batting_team',y='total_runs',data=Inning2_score)

plt.xlabel('Runs',fontsize=15)
plt.ylabel('Teams',fontsize=15)
plt.title('Inning 2 Scores',size=20)
plt.xticks(rotation=90)

plt.show()

**Number of times Team Scored 200+ per inning**

In [ ]:
scored_200=runs.groupby(['match_id','inning','batting_team','bowling_team'])['total_runs'].sum().reset_index()
scored_200=scored_200[scored_200['total_runs']>=200]


plt.figure(figsize=(7,5))
sns.countplot(scored_200['batting_team'])
plt.xticks(rotation=90)
plt.xlabel('Teams',fontsize=15)
plt.ylabel('Count',fontsize=15)
plt.title('No Of Times 200 Scored Batting First',size=20)
plt.figure(figsize=(7,5))
sns.countplot(scored_200['bowling_team'])
plt.xticks(rotation=90)
plt.xlabel('Teams',fontsize=15)
plt.ylabel('Count',fontsize=15)
plt.title('No Of Times 200 Scored Bowling First',size=20)
plt.show()

RCB holds the record of maxmimum 200+ score in 1st inning and KIXP  holds the record of maxmimum 200+ score in 2st inning

# Batsman Stats

In [ ]:
runs_scored=del_df.groupby('batsman')['batsman_runs'].sum().reset_index()
balls_played=del_df.groupby('batsman')['ball'].count().reset_index()
runs_scored=runs_scored.merge(balls_played,on='batsman',how='outer')
bats_man_runs=del_df.groupby('batsman').apply(lambda x:x['batsman_runs'].sum()).reset_index(name='Runs').sort_values(by='Runs',ascending=False)
bats_man_df=pd.DataFrame(bats_man_runs)
runs_series = del_df.groupby([del_df.match_id,del_df.batsman]).batsman_runs.sum()
runs_df = pd.DataFrame(runs_series)
runs_df = runs_df.reset_index()
condition = runs_df.batsman_runs >= 100
player_100df = runs_df.loc[condition,:]
centuries=player_100df.groupby(player_100df.batsman).batsman_runs.count().reset_index(name='centuries').sort_values(by='centuries',ascending=False)
runs_scored=runs_scored.merge(fours,on='batsman',how='outer')
runs_scored=runs_scored.merge(sixes,on='batsman',how='outer')
runs_scored=runs_scored.merge(centuries,on='batsman',how='outer')
runs_scored['strike_rate']=(runs_scored['batsman_runs']/runs_scored['ball'])*100
runs_scored=runs_scored.sort_values(by='batsman_runs',ascending=False)
high=del_df.groupby(['match_id','batsman','batting_team'])['batsman_runs'].sum().reset_index()
high=high.groupby(['batsman','batting_team'])['batsman_runs'].max().reset_index()
runs_scored=runs_scored.merge(high,on='batsman',how='outer')
top_scorer=del_df.groupby(['match_id','batting_team','batsman'])['batsman_runs'].sum().reset_index().sort_values(by='batsman_runs',ascending=False)
runs_scored.rename(columns={'batsman_runs_x':'runs','batsman_runs_y':'highest'},inplace=True)
runs_scored.fillna(0,inplace=True)

In [ ]:
def batsman_stat(batsman):
    return runs_scored[runs_scored['batsman']==batsman]

batsman_stat('V Kohli')

In [ ]:
top_scorer[:10].plot.barh(x='batsman',y='batsman_runs',figsize=(6,6),color="#E74C3C",title='Individual Top Scorers')
plt.xlabel('Runs',fontsize=15)
plt.ylabel('Batsman',fontsize=15)
plt.figure(figsize=(7,5))
sns.barplot(x='batsman',y='centuries',data=centuries[:10])
plt.xticks(rotation=90)
plt.xlabel('Batsman',fontsize=15)
plt.ylabel('Centuries',fontsize=15)
plt.title('Centuries Per Batsman',size=20)

plt.show()

Chris Gayle has record of highest runs in a match and 7 highest number of centuries(7)

# Now let's compare Batsman and bowler

In [ ]:
batsman_vs_bowler=del_df.groupby(['batsman','bowler']).apply(lambda x:x['batsman_runs'].sum()).reset_index(name='Runs_scored').sort_values(by='Runs_scored',ascending=False)
balls_faced=del_df.groupby(['batsman','bowler']).apply(lambda x:x['ball'].count()).reset_index(name='balls_faced').sort_values(by='balls_faced',ascending=False)
batsman_vs_bowler=batsman_vs_bowler.merge(balls_faced,on=['batsman','bowler'])
fours_=runs.groupby(['batsman','bowler'])['batsman_runs'].agg(lambda x:(x==4).sum()).reset_index(name='Fours')
sixes_=runs.groupby(['batsman','bowler'])['batsman_runs'].agg(lambda x:(x==6).sum()).reset_index(name='Sixes')
batsman_vs_bowler=batsman_vs_bowler.merge(fours_,on=['batsman','bowler'])
batsman_vs_bowler=batsman_vs_bowler.merge(sixes_,on=['batsman','bowler'])
wickets=del_df.groupby(['bowler','batsman']).apply(lambda x:x['player_dismissed'].count()).reset_index()
cond=['caught', 'bowled', 'lbw', 'caught and bowled','stumped', 'hit wicket']
wickets=del_df[del_df['dismissal_kind'].isin(cond)].groupby(['batsman','bowler'])['player_dismissed'].count().reset_index(name='wickets').sort_values(by='wickets',ascending=False)
batsman_vs_bowler=batsman_vs_bowler.merge(wickets,on=['batsman','bowler'])
batsman_vs_bowler['striker rate']=(batsman_vs_bowler['Runs_scored']/batsman_vs_bowler['balls_faced'])*100
def batsman_bowler_comparator(batsman,bowler):
    comp=batsman_vs_bowler[batsman_vs_bowler['batsman']==batsman][batsman_vs_bowler['bowler']==bowler]
    return comp


In [ ]:
batsman_bowler_comparator('V Kohli','JJ Bumrah')


Let's see the batsman ang their scoring ways.

In [ ]:
scoring_ways=del_df.groupby(['batsman','batsman_runs'])['total_runs'].count().reset_index()
scoring_ways=scoring_ways.pivot('batsman','batsman_runs','total_runs')
fig,ax=plt.subplots(2,2,figsize=(18,12))
scoring_ways[1].sort_values(ascending=False)[:5].plot(kind='barh',ax=ax[0,0],color='#80DEEA',width=0.8)
ax[0,0].set_title("Highest 1's",fontsize=15)
ax[0,0].set_ylabel('')
scoring_ways[2].sort_values(ascending=False)[:5].plot(kind='barh',ax=ax[0,1],color='#FFF176',width=0.8)
ax[0,1].set_title("Highest 2's",fontsize=15)
ax[0,1].set_ylabel('')
scoring_ways[4].sort_values(ascending=False)[:5].plot(kind='barh',ax=ax[1,0],color='#81C784',width=0.8)
ax[1,0].set_title("Highest 4's",fontsize=15)
ax[1,0].set_ylabel('')
scoring_ways[6].sort_values(ascending=False)[:5].plot(kind='barh',ax=ax[1,1],color='#EF5350',width=0.8)
ax[1,1].set_title("Highest 6's",fontsize=15)
ax[1,1].set_ylabel('')
plt.title('Top Scorer',size=20)
plt.show()

Chris Gayle holds record for maximum sixes

Shikhar Dhawan holds record for maxumum fours

Ms Dhoni for 2 runs and Virat Kohli for singles.

**Let's Check Out Orange cap holders**

In [ ]:
orange_holder=runs.groupby(['season','batsman']).apply(lambda x:x['batsman_runs'].sum()).reset_index(name='runs').sort_values(by='runs',ascending=False)
orange_holder=orange_holder.drop_duplicates(subset='season',keep='first').sort_values(by='season')
orange_holder.plot.bar(x='batsman',y='runs',figsize=(6,6),color="#FF9933",title='Orange Cap Holders(2008-2019)')
plt.xlabel('Batsman',fontsize=15)
plt.ylabel('Runs',fontsize=15)
plt.show()

# Runs Distribution of top 5 Batsman Per season.

In [ ]:
batsman_per_season=runs.groupby(['season','batsman'])['batsman_runs'].sum().reset_index()
batsman_per_season=batsman_per_season.groupby(['season','batsman'])['batsman_runs'].sum().unstack().T
batsman_per_season['Total']=batsman_per_season.sum(axis=1)
batsman_per_season=batsman_per_season.sort_values(by='Total',ascending=0)[:5]
batsman_per_season.drop('Total',axis=1,inplace=True)
batsman_per_season.T.plot(color=['#B71C1C','#311B92','#772272','green','#f0ff00'],marker='o')
fig=plt.gcf()
fig.set_size_inches(16,6)
plt.show()


In [ ]:
cond=['caught', 'bowled', 'lbw', 'caught and bowled','stumped', 'hit wicket']
wickets_per_bowler=del_df[del_df['dismissal_kind'].isin(cond)].groupby('bowler').apply(lambda x:x['player_dismissed'].count()).reset_index(name='wickets').sort_values(by='wickets',ascending=False)
balls_bowled=del_df.groupby('bowler').apply(lambda x:x['ball'].count()).reset_index(name='balls_bowled')
runs_given=del_df.groupby('bowler').apply(lambda x:x['total_runs'].sum()).reset_index(name='runs_given')
wickets_per_bowler=wickets_per_bowler.merge(balls_bowled,on='bowler')
wickets_per_bowler=wickets_per_bowler.merge(runs_given,on='bowler')
wickets_per_bowler['Economy']=wickets_per_bowler['runs_given']/(wickets_per_bowler['balls_bowled']/6)
max_in_match=runs[runs['dismissal_kind'].isin(cond)].groupby(['match_id','bowler']).apply(lambda x:x['player_dismissed'].count()).reset_index(name='Highest_in_match').sort_values(by=['bowler','Highest_in_match']).drop_duplicates(subset='bowler',keep='last')
wickets_per_bowler=wickets_per_bowler.merge(max_in_match,on='bowler')
wickets_per_bowler['Average']=wickets_per_bowler['runs_given']/wickets_per_bowler['wickets']
wickets_per_bowler.drop('match_id',axis=1,inplace=True)


In [ ]:
def bowler_stats(bowler):
    return wickets_per_bowler[wickets_per_bowler['bowler']==bowler]

bowler_stats('JJ Bumrah')

In [ ]:
wickets_per_bowler[:10].sort_values(by='wickets').plot.barh(x='bowler',y='wickets',figsize=(8,5),color="#33FFFF",title='Top 10 Wicket Taking Bowlers')
plt.xlabel('Bowler',fontsize=15)
plt.ylabel('Wickets',fontsize=15)
plt.show()

Lasith Malinga(Yorker Specialist) Has taken highest number of wickets.

# Purple Cap Holders

In [ ]:
purple_holder=runs[runs['dismissal_kind'].isin(cond)].groupby(['season','bowler']).apply(lambda x:x['player_dismissed'].count()).reset_index(name='wickets')
purple_holder=purple_holder.sort_values(by=['season','wickets']).drop_duplicates(subset='season',keep='last')
purple_holder.plot.bar(x='bowler',y='wickets',figsize=(6,6),color="#311B92",title='Purple Cap Holders(2008-2019)')
plt.xlabel('Bowler',fontsize=15)
plt.ylabel('Wickets',fontsize=15)
plt.show()

# Bolwers with some best figures

In [ ]:

wickets_per_bowler[:5].sort_values(by='wickets').plot.barh(x='bowler',y='wickets')
plt.xlabel('Bowler',fontsize=15)
plt.ylabel('Wickets',fontsize=15)
plt.title('Most Wicket Taker',size=20)

wickets_per_bowler[wickets_per_bowler['balls_bowled']//6>=300].sort_values(by='Economy')[:5].plot.barh(x='bowler',y='Economy')
plt.xlabel('Bowler',fontsize=15)
plt.ylabel('Economy',fontsize=15)
plt.title('Best Economy(More Than 300 Overs)',size=20)

wickets_per_bowler[wickets_per_bowler['balls_bowled']//6>=300].sort_values(by='Average',ascending=False)[:5].plot.barh(x='bowler',y='Average')
plt.xlabel('Bowler',fontsize=15)
plt.ylabel('Average',fontsize=15)
plt.title('Best Average(More Than 300 Overs)',size=20)
plt.show()

Highest number of wickets taken:Lasith Malinga

Best Economy:Dale Steyn

Best Average:Ishant Sharma

**Dismissal Types**

In [ ]:
dismiss_type=["run out","bowled", "caught", "lbw", "stumped", "caught and bowled", "hit wicket"]
wicket_type=del_df[del_df['dismissal_kind'].isin(dismiss_type)]
wicket_type=wicket_type['dismissal_kind'].value_counts().reset_index()
plt.figure(figsize=(6,6))
plt.pie(wicket_type['dismissal_kind'],labels=wicket_type['index'],autopct='%1.1f%%',shadow=True,colors=['#66CC99','#CC9999','#CCFF99','#B3E5FC','#B71C1C','#E59866'])

plt.show()

# Finals Analysis

let's find out the number of times team played final and won


In [ ]:
finals=match.drop_duplicates(subset=['season'],keep='last')
finals=finals[['id','season','city','team1','team2','toss_winner','toss_decision','winner']]
most_finals=pd.concat([finals['team1'],finals['team2']]).value_counts().reset_index()
most_finals.rename({'index':'team',0:'count'},axis=1,inplace=True)
xyz=finals['winner'].value_counts().reset_index()
most_finals=most_finals.merge(xyz,left_on='team',right_on='index',how='outer')
most_finals=most_finals.replace(np.NaN,0)
most_finals.drop('index',axis=1,inplace=True)
most_finals.set_index('team',inplace=True)
most_finals.rename({'count':'finals_played','winner':'win_count'},inplace=True,axis=1)

In [ ]:
most_finals.plot.bar(width=0.8,color=sns.color_palette("muted", 10))
plt.xlabel('Teams',size=15)
plt.gcf().set_size_inches(10,6)
plt.show()

Chennai Has played maximum IPL finals ,But Mumbai has won maximum IPL Finals

**Toss Decisions in Finals**

In [ ]:
finals['toss_and_match_win']=finals['toss_winner']==finals['winner']
finals['toss_and_match_win'].replace({True:'Yes',False:'No'},inplace=True)
sns.countplot(x='toss_decision',hue='toss_and_match_win',data=finals,palette=sns.color_palette("muted", 10))
plt.xlabel('Toss Decision',size=15)
plt.ylabel('Counts',size=15)
plt.title('Toss Decisions in Finals',size=20)
plt.show()

**Finals Won per Decision**

In [ ]:
plt.figure(figsize=(6,6))
sns.countplot(x='toss_decision',hue='toss_and_match_win',data=finals,palette=sns.color_palette("muted", 10))
plt.xlabel('Final Decision',fontsize=15)
plt.ylabel('Wins',fontsize=15)
plt.title('Wins Per Decision',size=20)
plt.show()

Let's see if toss winner wins the final

In [ ]:
final_toss_match_winner=finals[finals['toss_winner']==finals['winner']]
slices=[len(final_toss_match_winner),(12-len(final_toss_match_winner))]
labels=['yes','no']
plt.pie(slices,labels=labels,startangle=90,shadow=True,colors=['#41A317','#E41B17'],explode=(0,0.05),autopct='%1.1f%%')
fig = plt.gcf()
fig.set_size_inches(5,5)
plt.show()

There's 75% chances of winning the final match on toss win.